# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,75.462702,0.243796,1234,75.706498,75.218906,151.412996,150.437812
6,801.707270,0.368433,1234,802.075704,801.338837,1604.151407,1602.677675
7,127.441196,-0.850108,1234,126.591087,128.291304,253.182175,256.582608
8,610.874899,0.369348,1234,611.244247,610.505551,1222.488494,1221.011103
9,199.396910,0.307498,1234,199.704409,199.089412,399.408817,398.178824
10,531.935515,0.286823,1234,532.222338,531.648693,1064.444675,1063.297385
11,612.659871,0.538001,1234,613.197872,612.121870,1226.395745,1224.243739
12,782.245900,0.366954,1234,782.612854,781.878946,1565.225708,1563.757891
13,741.761636,-0.797030,1234,740.964606,742.558666,1481.929212,1485.117332
14,439.395927,0.070273,1234,439.466200,439.325654,878.932400,878.651309


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-75.706498,75.218906
6,-802.075704,801.338837
7,-126.591087,128.291304
8,-611.244247,610.505551
9,-199.704409,199.089412
10,-532.222338,531.648693
11,-613.197872,612.121870
12,-782.612854,781.878946
13,-740.964606,742.558666
14,-439.466200,439.325654


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-75.706498,75.218906
6,-802.075704,801.338837
7,-126.591087,128.291304
8,-611.244247,610.505551
9,-199.704409,199.089412
10,-532.222338,531.648693
11,-613.197872,612.121870
12,-782.612854,781.878946
13,-740.964606,742.558666
14,-439.466200,439.325654


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[ -75.70649785,   75.21890597],
       [-802.07570353,  801.33883729],
       [-126.59108737,  128.29130405],
       [-611.24424705,  610.50555138],
       [-199.70440853,  199.08941212],
       [-532.22233772,  531.64869253],
       [-613.19787235,  612.12186974],
       [-782.61285383,  781.87894573],
       [-740.96460609,  742.55866593],
       [-439.4662001 ,  439.32565435]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,149.412996,-150.437812
6,1602.151407,-1602.677675
7,251.182175,-256.582608
8,1220.488494,-1221.011103
9,397.408817,-398.178824
10,1062.444675,-1063.297385
11,1224.395745,-1224.243739
12,1563.225708,-1563.757891
13,1479.929212,-1485.117332
14,876.932400,-878.651309


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
